In [1]:
import pandas as pd

orders = pd.read_csv("orders.csv")
print(orders.head())

   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  


In [2]:
users = pd.read_json("users.json")
print(users.head())

   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold


In [3]:
import sqlite3

conn = sqlite3.connect(":memory:")
with open("restaurants.sql", "r") as f:
    conn.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)
print(restaurants.head())

   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [4]:
order_user_df = orders.merge(
    users,
    on="user_id",
    how="left"
)

In [5]:
final_df = order_user_df.merge(
    restaurants,
    on="restaurant_id",
    how="left"
)

In [6]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)

In [7]:
final_df.head()

,order_id,user_id,restaurant_id,order_date,total_amount,restaurant_name_x,name,city,membership,restaurant_name_y,cuisine,rating
0,1,2508,450,18-02-2023,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,18-01-2023,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,15-07-2023,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,04-10-2023,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,25-12-2023,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [8]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])

final_df.groupby(final_df["order_date"].dt.month)["order_id"].count()

/tmp/ipython-input-4028215665.py:1: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  final_df["order_date"] = pd.to_datetime(final_df["order_date"])


,order_id
order_date,
1,831
2,785
3,903
4,812
5,844
6,784
7,859
8,851
9,812


In [10]:
gold_city_revenue = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .sort_values(ascending=False)
)

gold_city_revenue.head(1)

,total_amount
city,
Chennai,1080909.79


In [11]:
final_df.groupby("cuisine")["total_amount"].mean().sort_values(ascending=False).head(1)

,total_amount
cuisine,
Mexican,808.021344


In [12]:
user_total = final_df.groupby("user_id")["total_amount"].sum()

high_value_users = user_total[user_total > 1000]

high_value_users.count()

np.int64(2544)

In [15]:
bins = [3.0, 3.5, 4.0, 4.5, 5.0]
labels = [
    "3.0 – 3.5",
    "3.6 – 4.0",
    "4.1 – 4.5",
    "4.6 – 5.0"
]

final_df["rating_range"] = pd.cut(
    final_df["rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

In [16]:
final_df["rating_range"].value_counts().sort_index()

,count
rating_range,
3.0 – 3.5,2679
3.6 – 4.0,2118
4.1 – 4.5,2431
4.6 – 5.0,2772


In [17]:
final_df.groupby("rating_range")["total_amount"] \
    .sum() \
    .sort_values(ascending=False)

/tmp/ipython-input-1201112070.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  final_df.groupby("rating_range")["total_amount"] \


,total_amount
rating_range,
4.6 – 5.0,2197030.75
3.0 – 3.5,2136772.70
4.1 – 4.5,1960326.26
3.6 – 4.0,1717494.41


In [19]:
final_df[final_df["membership"] == "Gold"] \
    .groupby("city")["total_amount"] \
    .mean() \
    .sort_values(ascending=False) \
    .head(1)

,total_amount
city,
Chennai,808.45908


In [20]:
cuisine_restaurants = final_df.groupby("cuisine")["restaurant_id"].nunique()
cuisine_revenue = final_df.groupby("cuisine")["total_amount"].sum()

cuisine_analysis = pd.DataFrame({
    "restaurant_count": cuisine_restaurants,
    "total_revenue": cuisine_revenue
})

cuisine_analysis.sort_values(
    by=["restaurant_count", "total_revenue"],
    ascending=[True, False]
).head(1)

,restaurant_count,total_revenue
cuisine,,
Chinese,120,1930504.65


In [21]:
gold_orders = final_df[final_df["membership"] == "Gold"]["order_id"].count()
total_orders = final_df["order_id"].count()

percentage = round((gold_orders / total_orders) * 100)
percentage

50

In [25]:
restaurant_stats = final_df.groupby("restaurant_name_x").agg(
    avg_order_value=("total_amount", "mean"),
    order_count=("order_id", "count")
)

restaurant_stats[
    restaurant_stats["order_count"] < 20
].sort_values("avg_order_value", ascending=False).head(1)

,avg_order_value,order_count
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13


In [26]:
final_df.groupby(["city", "cuisine"])["total_amount"] \
    .sum() \
    .sort_values(ascending=False) \
    .head(1)

,,total_amount
city,cuisine,
Bangalore,Mexican,571004.61


In [27]:
allowed_combinations = [
    ("Gold", "Indian"),
    ("Gold", "Italian"),
    ("Regular", "Indian"),
    ("Regular", "Chinese")
]

filtered_df = final_df[
    final_df[["membership", "cuisine"]]
    .apply(tuple, axis=1)
    .isin(allowed_combinations)
]

filtered_df.groupby(
    ["membership", "cuisine"]
)["total_amount"].sum().sort_values(ascending=False)

,,total_amount
membership,cuisine,
Gold,Italian,1005779.05
Regular,Indian,992100.27
Gold,Indian,979312.31
Regular,Chinese,952790.91


In [28]:
final_df["order_date"] = pd.to_datetime(final_df["order_date"])
final_df["quarter"] = final_df["order_date"].dt.to_period("Q")

final_df.groupby("quarter")["total_amount"].sum().sort_values(ascending=False).head(1)

,total_amount
quarter,
2023Q3,2037385.1


In [29]:
gold_orders_count = final_df[final_df["membership"] == "Gold"]["order_id"].count()
gold_orders_count

np.int64(4987)

In [30]:
hyderabad_revenue = round(
    final_df[final_df["city"] == "Hyderabad"]["total_amount"].sum()
)

hyderabad_revenue

1889367

In [31]:
distinct_users = final_df["user_id"].nunique()
distinct_users

2883

In [32]:
avg_gold_order_value = round(
    final_df[final_df["membership"] == "Gold"]["total_amount"].mean(),
    2
)

avg_gold_order_value

np.float64(797.15)

In [33]:
high_rating_orders = final_df[final_df["rating"] >= 4.5]["order_id"].count()
high_rating_orders

np.int64(3374)

In [34]:
top_gold_city = (
    final_df[final_df["membership"] == "Gold"]
    .groupby("city")["total_amount"]
    .sum()
    .idxmax()
)

In [35]:
orders_in_top_gold_city = final_df[
    (final_df["membership"] == "Gold") &
    (final_df["city"] == top_gold_city)
]["order_id"].count()

orders_in_top_gold_city

np.int64(1337)